In [2]:
import requests
import json
import os
import sqlalchemy
import pandas as pd
import mdapi_functions as md
from IPython.display import display, Markdown
import markdown
from bokeh.plotting import figure, show
from bokeh.models import (Span, TabPanel, Tabs, ColumnDataSource, DataCube,
                          GroupingInfo, StringFormatter, SumAggregator,
                          TableColumn, HoverTool)
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.io import output_notebook
from math import pi
from datetime import datetime
import datacite_api_functions as dcf
import numpy as np
import data_doc_helper as dh

from math import pi


In [ ]:


output_notebook(hide_banner=True)

x = {
    'United States': 157,
    'United Kingdom': 93,
    'Japan': 89,
    'China': 63,
    'Germany': 44,
    'India': 42,
    'Italy': 40,
    'Australia': 35,
    'Brazil': 32,
    'France': 31,
    'Taiwan': 31,
    'Spain': 29,
}

data = pd.Series(x).reset_index(name='value').rename(columns={'index': 'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['color'] = Category20c[len(x)]

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [3]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["ethnicity"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [4]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["sex"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)


In [8]:
output_notebook(hide_banner=True)

df = md.get_freeze_profile()
df = df[df["demo_grp"].isin(["year_of_birth"])]
df2 = df[["cat", "n"]]
df2["angle"] = df2["n"]/ df2["n"].sum() * 2*pi
df2['color'] = Category20c[df2.__len__()]
df2["tot"] = df2["n"].sum()
df2

p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@cat: @n / @tot", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='cat', source=df2)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
show(p)
